<a href="https://colab.research.google.com/github/ounospanas/AIDL_B02/blob/main/AIDL_B02_REINFORCE_Pong_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assau

In [2]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
""" This is a pytorch implemetation of this https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5"""

import numpy as np
import time
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from matplotlib import pylab as plt
%matplotlib inline

In [3]:
#create policy network
class PolicyGradientNet(nn.Module):
    def __init__(self):
        super(PolicyGradientNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.head = nn.Linear(1568, 6)

    def forward(self, x):
        x = F.relu(self.bn1((self.conv1(x))))
        x = F.relu(self.bn2((self.conv2(x))))
        x = F.relu(self.bn3((self.conv3(x))))
        return F.softmax(self.head(x.view(x.size(0), -1)), dim=1)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
model = PolicyGradientNet().to(device)

In [6]:
learning_rate = 0.0001
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [7]:
def loss_fn(preds, r): 
    # pred is output from neural network, a is action index
    # r is return (sum of rewards to end of episode), d is discount factor
    r = torch.Tensor(r)
    return torch.sum(-r * torch.log(preds)) # element-wise multipliy, then sum

In [8]:
# hyperparameters
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

In [9]:
def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float)

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
env = gym.make("PongDeterministic-v4")
observation = env.reset()
prev_x = None # used in computing the difference frame
dlogps,drs = [],[]
running_reward = None
reward_sum = 0
episode_number = 0
drs = []
super_reward_sum = 0
time_steps = []
training_epochs = 2000
MAX_DUR = 10000
load = False
count = 0

if load:
    model = torch.load('pong_pytorch.pt')

for _ in range(training_epochs):
    
    if render: env.render() # in case you run the code in a local machine
    curr_state = env.reset()
    transitions = [] # list of state, action, rewards
    done = False
    
    for t in range(MAX_DUR): #while in episode
        
        # preprocess the observation, set input to network to be difference image
        cur_x = prepro(observation)
        x = cur_x - prev_x if prev_x is not None else np.zeros_like(cur_x)
        prev_x = cur_x
        x = torch.from_numpy(x)
        x = x.view(1,1,80,80)
    
        # forward the policy network and sample an action from the returned probability
        aprob = model(x.to(device, dtype=torch.float))
        m = Categorical(aprob)
        action = m.sample()
        action = action.item()

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        #append state, action, reward tuple in the transitions
        transitions.append((x, aprob[0][action], reward))
        
        if done:
            break

    # Optimize policy network with full episode
    ep_len = len(transitions) # episode length
    time_steps.append(ep_len)
    preds = torch.zeros(ep_len)
    discounted_rewards = torch.zeros(ep_len)

    count+=1
    rewards = []
    for i in range(ep_len): #for each step in episode
        
        state, action, reward = transitions[i]
        preds[i] = action
        rewards.append(reward)
    
    epr = np.vstack(rewards)

    # normalize rewards
    discounted_rewards = discount_rewards(epr)
    discounted_rewards -= np.mean(discounted_rewards)
    eps = np.finfo(np.float32).eps.item()
    discounted_rewards /= np.std(discounted_rewards) +eps
    
    #compute loss
    policy_loss = -torch.sum(torch.Tensor(discounted_rewards) * torch.log(preds).view(-1,1))
    loss = policy_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print("For episode {} the reward is: {}".format(count,reward_sum))
    super_reward_sum += reward_sum
    reward_sum = 0
    
    if count%100==0:
        print('The mean of last 100 rewards is {}'.format(super_reward_sum/100))
        super_reward_sum = 0
        torch.save(model, 'pong_pytorch.pt') 
        
env.close()

For episode 1 the reward is: -18.0
For episode 2 the reward is: -21.0
For episode 3 the reward is: -21.0
For episode 4 the reward is: -20.0
For episode 5 the reward is: -21.0
For episode 6 the reward is: -21.0
For episode 7 the reward is: -21.0
For episode 8 the reward is: -20.0
For episode 9 the reward is: -21.0
For episode 10 the reward is: -20.0
For episode 11 the reward is: -20.0
For episode 12 the reward is: -20.0
For episode 13 the reward is: -21.0
For episode 14 the reward is: -20.0
For episode 15 the reward is: -20.0
For episode 16 the reward is: -20.0
For episode 17 the reward is: -21.0
For episode 18 the reward is: -19.0
For episode 19 the reward is: -20.0
For episode 20 the reward is: -19.0
For episode 21 the reward is: -18.0
For episode 22 the reward is: -19.0
For episode 23 the reward is: -20.0
For episode 24 the reward is: -20.0
For episode 25 the reward is: -21.0
For episode 26 the reward is: -20.0
For episode 27 the reward is: -20.0
For episode 28 the reward is: -21.0
F

In [11]:
from gym import wrappers
env = gym.make("PongDeterministic-v4")
env = wrappers.Monitor(env, "./gym-results", force=True)

model = torch.load('pong_pytorch.pt')
state = env.reset()
reward_sum = 0
prev_x = None

for t in range(10000):
    cur_x = prepro(state)
    x = cur_x - prev_x if prev_x is not None else np.zeros_like(cur_x)
    prev_x = cur_x
    x = torch.from_numpy(x)
    x = x.view(1,1,80,80)
    model.eval()
    with torch.no_grad():
        aprob = model(x.to(device, dtype=torch.float))
    #print(aprob)
    m = Categorical(aprob)
    action = m.sample()
    #self.saved_log_probs.append(m.log_prob(action))
    action = action.item()
    #env.render()
    state, reward, done, _ = env.step(action)
    reward_sum += reward
    
    if done:
        break 
print(reward_sum)
env.close()

17.0


In [12]:
import io
import base64
from IPython.display import HTML

video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))